In [5]:
import pandas as pd
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [8]:
tabelas = pd.read_html("https://mudancanorteblumenau.paginas.ufsc.br/logistica/area-dos-lideres-de-ambiente/lista-de-ambientes-e-lideres-designados/")

In [14]:
DADOS = []

for tabela in tabelas:
    edificio = tabela.iloc[0,0]
    cabecalho = tabela.iloc[1,:]
    dados = tabela.iloc[2:,:]
    dados.columns = cabecalho
    dados["edificio"] = edificio
    DADOS.append(dados)

/tmp/ipykernel_1447113/2592225809.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados["edificio"] = edificio
/tmp/ipykernel_1447113/2592225809.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados["edificio"] = edificio
/tmp/ipykernel_1447113/2592225809.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

In [15]:
DADOS = pd.concat(DADOS)

In [23]:
DADOS["a_codigo"] = DADOS.Ambiente.apply(lambda x: x.split(" – ")[0].strip())

In [25]:
DADOS.rename(columns={"Líder de Ambiente": "lider"}, inplace=True)

In [35]:
lideres = DADOS.lider.unique()

In [40]:
DATABASE = {}

for lider in lideres:
    DATABASE[lider] = list(DADOS.loc[DADOS.lider == lider, "a_codigo"]) 

In [42]:
import json

with open("json/lideres.json", "w") as f:
    json.dump(DATABASE, f, indent=2, ensure_ascii=False)